In [ ]:
# ensure that the product of 

In [1]:
import numpy as np
import pandas as pd
import scipy
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

from sklearn.ensemble import RandomForestClassifier
from scipy.optimize import curve_fit

from scipy.stats import boxcox
from multiprocessing import Pool, cpu_count
import tqdm
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import re, string, timeit
from xgboost import plot_importance
from gensim.models import Word2Vec 
sns.set(style="darkgrid")

%matplotlib inline

C:\Users\EweiwiA\Programs\anaconda\envs\kaggle\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\EweiwiA\Programs\anaconda\envs\kaggle\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
items = pd.read_csv('items.csv')
item_cat = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
train = pd.read_csv('sales_train_v2.csv')
test = pd.read_csv('test.csv')

In [3]:
train = train[train['item_price'] < 10000]
train = train[train['item_cnt_day'] < 600]

train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [4]:
item_cat = item_cat.sort_values(by=['item_category_name'])
item_cat['item_cat_level1'] = item_cat['item_category_name'].str.split('-').apply(lambda x: x[0].strip())


def func(x):
    try:
        return x[1]
    except Exception as e:
        return 'unknown'
item_cat['item_cat_level2'] = item_cat['item_category_name'].str.split('-').apply(lambda x: func(x)).value_counts()

item_cat.sort_values(by=['item_cat_level1', 'item_cat_level2'], inplace=True)
item_cat['item_cat_level1'] = LabelEncoder().fit_transform(item_cat['item_cat_level1'])
item_cat['item_cat_level2'] = LabelEncoder().fit_transform(item_cat['item_cat_level2'])

#item_cat['item_category_name'] = item_cat['item_category_name'].factorize()[0]

In [5]:
shops = shops.sort_values(by=['shop_name'])

shops['shop_city'] = shops['shop_name'].str.split(' ').apply(lambda x: x[0].strip())
shops.loc[(shops['shop_city'] == u'!Якутск'), 'shop_city'] = u'Якутск'
shops['shop_type']  = shops['shop_name'].str.split(' ').apply(lambda x: '_'.join([re.sub(r'[^\w\s]','',t) for t in x[1:2]]))
shops['shop_lname'] = shops['shop_name'].str.split(' ').apply(lambda x: '_'.join([re.sub(r'[^\w\s]','',t) for t in x[2:]]))

shops['shop_city'] = LabelEncoder().fit_transform(shops['shop_city'])
shops['shop_type'] = LabelEncoder().fit_transform(shops['shop_type'])
shops['shop_lname'] = LabelEncoder().fit_transform(shops['shop_lname'])

In [6]:
shops.head()

,shop_name,shop_id,shop_city,shop_type,shop_lname
0,"!Якутск Орджоникидзе, 56 фран",0,29,2,4
1,"!Якутск ТЦ ""Центральный"" фран",1,29,9,50
2,"Адыгея ТЦ ""Мега""",2,0,9,27
3,"Балашиха ТРК ""Октябрь-Киномир""",3,1,7,33
4,"Волжский ТЦ ""Волга Молл""",4,2,9,15


In [7]:
offers_df = pd.merge(train, shops, left_on=['shop_id'], right_on=['shop_id'], how='left')

offer_item_shop_times = offers_df.groupby(['item_id', 'shop_id'], as_index=False)['item_price'].count()
offer_item_shop_times.columns = ['item_id', 'shop_id', 'offer_item_shop_times']

offer_item_city_times = offers_df.groupby(['item_id', 'shop_city'], as_index=False)['item_price'].count()
offer_item_city_times.columns = ['item_id', 'shop_id', 'offer_item_city_times']

offer_shop_city_times = offers_df.groupby(['shop_id', 'shop_city'], as_index=False)['item_price'].count()
offer_shop_city_times.columns = ['item_id', 'shop_id', 'offer_shop_city_times']

In [8]:
days_per_month = pd.Series({0:31, 1:28, 2: 31, 3: 30, 4: 31, 5: 30, 6: 31, 7: 31, 8: 30, 9: 31, 10: 30, 11: 30})

In [9]:
matrix = train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'item_cnt_day': 'sum'})
matrix.columns = ['date_block_num', 'shop_id', 'item_id', 'item_cnt_month']
matrix['item_cnt_month'] = matrix['item_cnt_month'].clip(0,20)



In [10]:
###### test_df['item_cnt_month'] = 0
test['date_block_num'] = 34
test['item_cnt_month'] = 0 
test.set_index('ID', inplace=True)
test = test[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month']]


In [ ]:
matrix.head()

In [ ]:
test.head()

In [11]:
cols = ['date_block_num','shop_id','item_id']

matrix = pd.concat([matrix, test], axis=0, sort=False, ignore_index=True, keys=cols)

In [12]:
matrix.shape

(1814574, 4)

In [ ]:
#Unfortunately, offers does not help much
"""
matrix = pd.merge(matrix, offer_item_shop_times, left_on=['item_id', 'shop_id'], right_on= ['item_id', 'shop_id'], how='left')
matrix['offer_item_shop_times'].fillna(0, inplace=True)

matrix = pd.merge(matrix, offer_item_city_times, left_on=['item_id', 'shop_id'], right_on= ['item_id', 'shop_id'], how='left')
matrix['offer_item_city_times'].fillna(0, inplace=True)

matrix = pd.merge(matrix, offer_shop_city_times, left_on=['item_id', 'shop_id'], right_on= ['item_id', 'shop_id'], how='left')
matrix['offer_shop_city_times'].fillna(0, inplace=True)
"""

In [ ]:
print (matrix.shape)

matrix = pd.merge(matrix, shops, left_on='shop_id', right_on='shop_id', how='left')
print (matrix.shape)
matrix = pd.merge(matrix, items, left_on='item_id', right_on='item_id', how='left')
print (matrix.shape)
matrix = pd.merge(matrix, item_cat, left_on='item_category_id', right_on='item_category_id', how='left')
print (matrix.shape)
matrix['month'] =  (matrix['date_block_num'])%12
matrix['days_per_month'] = matrix['month'].map(days_per_month)

In [ ]:
matrix.info()

In [ ]:
matrix['date_block_num'].dtype

In [ ]:
matrix.date_block_num = matrix.date_block_num.astype('float16')
matrix.item_cnt_month = matrix.item_cnt_month.astype('int16')
matrix.item_id = matrix.item_id.astype('int16')
matrix.shop_id = matrix.shop_id.astype('int16')
matrix.item_category_id = matrix.item_category_id.astype('float16')
matrix.days_per_month = matrix.days_per_month.astype('float16')
matrix.item_cat_level1 = matrix.item_cat_level1.astype('float16')
matrix.item_cat_level2 = matrix.item_cat_level2.astype('float16')


In [ ]:
# construct item_id, shop_id sales matrix
sales_matrix = matrix.groupby(['item_id', 'shop_id'])['item_cnt_month'].sum().unstack('shop_id')

shop_id_sales_me_na = sales_matrix.mean(axis=0)
shop_id_sales_me_z = sales_matrix.fillna(0).mean(axis=0)
item_id_sales_me_na = sales_matrix.mean(axis=1)
item_id_sales_me_z = sales_matrix.fillna(0).mean(axis=1)


# mean encoding for shop id based on sales
matrix['m_shop_id_sales_me_na'] = matrix['shop_id'].map(shop_id_sales_me_na)
matrix['m_shop_id_sales_me_z'] = matrix['shop_id'].map(shop_id_sales_me_z)

# mean encoding for item_id based on sales
matrix['m_item_id_sales_me_na'] = matrix['item_id'].map(item_id_sales_me_na)
matrix['m_item_id_sales_me_z'] = matrix['item_id'].map(item_id_sales_me_z)

del sales_matrix
del shop_id_sales_me_na
del shop_id_sales_me_z
del item_id_sales_me_na
del item_id_sales_me_z

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
matrix = lag_feature(matrix, [1], 'item_cnt_month')

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'shop_id', 'mean_shop_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'shop_id'], right_on=['date_block_num', 'shop_id'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1,2,3], 'mean_shop_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_category_id'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'item_category_id', 'mean_item_cat_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'item_category_id'], right_on=['date_block_num', 'item_category_id'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_item_cat_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'item_id', 'mean_item_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'item_id'], right_on=['date_block_num', 'item_id'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1,2,3], 'mean_item_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'item_id', 'shop_city', 'mean_item_city_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'item_id', 'shop_city'], right_on=['date_block_num', 'item_id', 'shop_city'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_item_city_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'item_id', 'shop_type', 'mean_item_shop_type_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'item_id', 'shop_type'], right_on=['date_block_num', 'item_id', 'shop_type'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_item_shop_type_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id', 'shop_lname'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'item_id', 'shop_lname', 'mean_item_shop_lname_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'item_id', 'shop_lname'], right_on=['date_block_num', 'item_id', 'shop_lname'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_item_shop_lname_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'shop_city'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'shop_id', 'shop_city', 'mean_shop_city_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'shop_id', 'shop_city'], right_on=['date_block_num', 'shop_id', 'shop_city'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1,2,3], 'mean_shop_city_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_type'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'shop_type', 'mean_shop_type_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'shop_type'], right_on=['date_block_num', 'shop_type'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_shop_type_sales')
print (matrix.shape)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_lname'], as_index=False)['item_cnt_month'].mean()
group.columns = ['date_block_num', 'shop_lname', 'mean_shop_lname_sales']
matrix = pd.merge(matrix, group, left_on = ['date_block_num', 'shop_lname'], right_on=['date_block_num', 'shop_lname'], how='left')
print (matrix.shape)

matrix = lag_feature(matrix, [1], 'mean_shop_lname_sales')
print (matrix.shape)

In [ ]:
matrix.drop(['shop_name', 'item_name', 'item_category_name'], axis=1, inplace=True)

In [ ]:
#matrix = matrix[matrix.date_block_num > 11]

In [ ]:
matrix.head()

In [ ]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)
        else:
            
            df[col].fillna(df[col].mean(), inplace=True)
    return df

tmp = fill_na(matrix)

In [ ]:
X_train = matrix[ (matrix.date_block_num < 33)].drop(['item_cnt_month'], axis=1)
Y_train = matrix[ (matrix.date_block_num < 33)]['item_cnt_month']


X_valid = matrix[(matrix.date_block_num == 33) ].drop(['item_cnt_month'], axis=1)
Y_valid = matrix[(matrix.date_block_num == 33) ]['item_cnt_month']


X_test = matrix[matrix.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
model = XGBRegressor(
    max_depth=5,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.8, 
    subsample=0.8,
    eta=0.3,
    reg_lambda=1,
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 5)


In [ ]:
if pd.Series(Y_test).max() < 5:
    print ('something wrong going on with the data you have')

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(model, (10,14))


In [ ]:
Y_pred = model.predict(X_valid).clip(0,20)
Y_test = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission_mine.csv', index=False)

In [ ]:
Y_train.describe()

In [ ]:
pd.Series(Y_pred).mean()

In [ ]:
from fastai.tabular import *
import pandas as pd
from fastai.datasets import *

In [ ]:
cat_names = ['item_id', 
            'shop_id', 
            'city', 'shop_type',
            'shop_lname', 
            'item_category_id', 
            'item_cat_level1', 
            'item_cat_level2']
    
cont_names = ['date_block_num',  
            'month', 
            'days_per_month', 
            'm_shop_id_sales_me_na',
            'm_shop_id_sales_me_z',
            'm_item_id_sales_me_na', 
            'm_item_id_sales_me_z',
            'item_cnt_month_lag_1',
            'mean_shop_sales', 
            'mean_shop_sales_lag_1',
            'mean_shop_sales_lag_2', 
            'mean_shop_sales_lag_3', 
            'mean_item_cat_sales',
            'mean_item_cat_sales_lag_1', 
            'mean_item_sales', 
            'mean_item_sales_lag_1',
            'mean_item_sales_lag_2', 
            'mean_item_sales_lag_3',
            'mean_item_city_sales', 
            'mean_item_city_sales_lag_1',
            'mean_item_shop_type_sales', 
            'mean_item_shop_type_sales_lag_1',
            'mean_item_shop_lname_sales', 
            'mean_item_shop_lname_sales_lag_1',
            'mean_shop_city_sales', 
            'mean_shop_city_sales_lag_1',
            'mean_shop_city_sales_lag_2', 
            'mean_shop_city_sales_lag_3',
            'mean_shop_type_sales', 
            'mean_shop_type_sales_lag_1',
            'mean_shop_lname_sales', 
            'mean_shop_lname_sales_lag_1']



In [ ]:
bs = 64
procs = [FillMissing, Categorify, Normalize]

In [ ]:
X_train['label'] = Y_train
X_valid['label'] = Y_valid
df = pd.concat([X_train, X_valid], axis=0)

In [ ]:
df.shape[0]

In [ ]:
test = TabularList.from_df(X_test, path='.', cat_names=cat_names, cont_names=cont_names)

In [ ]:
X_test.reset_index(inplace=True)

In [ ]:
from fastai import *
doc(FloatList)

In [ ]:
X_train.shape[0]

In [ ]:
df.shape[0]

In [ ]:
data = (TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs,)
                .split_by_idx(list(range(X_train.shape[0],df.shape[0])))
                .label_from_df(cols='label', label_cls=FloatList, log=True)
                .add_test(test)
                .databunch())

In [ ]:
data.show_batch(rows=10)

In [ ]:
defaults.device

In [ ]:
max_log_y = np.log(np.max(df['label']))
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [ ]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [ ]:
learn.fit(1, 1e-2)

In [ ]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
!which python

In [ ]:
!where python

In [ ]:
X_test.head()